![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=Vertex+Grounding+Story.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/Vertex%20Grounding%20Story.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FVertex%2520Grounding%2520Story.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/Vertex%20Grounding%20Story.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/Vertex%20Grounding%20Story.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI - Ground Gemini

[Grounding Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview)


---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [4]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [5]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [6]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'grounding-story'

packages:

In [8]:
from google.cloud import aiplatform
import vertexai.generative_models # for Gemini Models
import IPython

In [9]:
aiplatform.__version__

'1.48.0'

clients:

In [10]:
vertexai.init(project = PROJECT_ID, location = REGION)

---
## Vertex AI Package

With the [vertexai](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai) client there are packages for the types of data being interacted with.  There is also a higher package for preview models (not yet in GA).

> **NOTE:** In can be helpful to review the API Documentation at it's source in GitHub for up to the moment release information: [github/googleapis/python-aiplatform](https://github.com/googleapis/python-aiplatform/tree/main)

Gemini Text and Multimodal Models:
- [vertexai.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models)
    - [vertexai.preview.generative_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.generative_models)

Language Models (PaLM and Codey Models):
- [vertexai.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models)
    - [vertexai.preview.language_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models)

Vision Models (Imagen Models):
- [vertexai.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.vision_models)
    - [vertexai.preview.vision_models()](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.vision_models)


In [11]:
# Gemini Models
gemini_text = vertexai.generative_models.GenerativeModel("gemini-1.0-pro")
#gemini_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.0-pro-vision")
#gemini_multimodal15 = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-preview-0409")

---
## Gemini Models

[Reference: Gemini API](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini)

### Text Prompt

In [12]:
prompt = "What material and size are bases?"

In [13]:
response = gemini_text.generate_content(
    prompt
)
IPython.display.Markdown(response.text)

## Baseball Base Material and Size

### Material

* **Professional:** Made of white rubber covered in horsehide or cowhide leather.
* **Amateur:** Often made of synthetic materials like polyurethane or rubber.
* **Youth:** Can be made from a variety of materials, including plastic, rubber, and foam.

### Size

* **Regulation:** 15 inches (38.1 cm) square.
* **Youth:** Smaller sizes are available for younger players, typically 12 inches (30.5 cm) or 14 inches (35.6 cm) square.

**Additional Information:**

* The bases are filled with a soft material like cork or kapok.
* The bottom of the base is slightly rounded to prevent it from tipping over easily.
* The first and third bases are sometimes equipped with a small "lip" to help fielders catch throws.


## Softball Base Material and Size

### Material

* **Regulation:** Made of canvas filled with a soft material like cork or kapok.
* **Practice:** Can be made from a variety of materials, including plastic, rubber, and foam.

### Size

* **Regulation:** 15 inches (38.1 cm) square.
* **Youth:** Smaller sizes are available for younger players, typically 12 inches (30.5 cm) or 14 inches (35.6 cm) square.

**Additional Information:**

* Softball bases are typically white or yellow.
* The first and third bases are sometimes equipped with a small "lip" to help fielders catch throws.


### Text Prompt: Ground With Vertex AI Search

[Grounding Gemini to your data](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini#private-ground-gemini)

In [14]:
from vertexai.preview.generative_models import GenerationConfig, grounding, Tool

In [15]:
prompt = "What material and size are bases?"

In [16]:
VS_LOCATION = 'global'
VS_DATASTORE_ID = 'mlb-latest_1713794093125'
response = gemini_text.generate_content(
    prompt,
    tools = [Tool.from_retrieval(
        grounding.Retrieval(grounding.VertexAISearch(
            datastore = f"projects/{PROJECT_ID}/locations/{VS_LOCATION}/collections/default_collection/dataStores/{VS_DATASTORE_ID}"
        ))
    )],
    generation_config = GenerationConfig(
        temperature = 0.0
    )
)
IPython.display.Markdown(response.text)

First, second, and third bases are marked by white canvas or rubber-covered bags, securely attached to the ground. The bags are 18 inches square, not less than three nor more than five inches thick, and filled with soft material. 

**Disclaimer:** This information is based on the 2024 Official Baseball Rules. For the most up-to-date information, please refer to the official MLB website.

### Text Prompt: Grounding Using Google Search

[Grounding With Google Search](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini#web-ground-gemini)

In [17]:
from vertexai.preview.generative_models import GenerationConfig, grounding, Tool

In [18]:
prompt = "What material and size are bases in Major League Baseball?"

In [19]:
response = gemini_text.generate_content(
    prompt,
    tools = [Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())],
    generation_config=GenerationConfig(
        temperature = 0.0
    )
)
IPython.display.Markdown(response.text)

Major League Baseball bases are made of white canvas or rubber and are 18 inches square. The bases are filled with soft material and are securely attached to the ground.